# 実習9
#### 時系列データ解析

#### 参考
PythonとKerasによるディープラーニング
(https://www.amazon.co.jp/PythonとKerasによるディープラーニング-Francois-Chollet/dp/4839964262)　6章

気象時系列データセットを使用する。これはドイツのイエナにあるMax Planck Institute for Biogeochemistryの観測所で記録されたデータである。このデータセットは14種類の数値(気温、気圧、湿度、風向など)を10分おきに記録した数年分のデータで構成されている。最も古いデータは2003年のものだが、ここでは2009年から2016年のデータのみを使用する。このデータを使用して、最近の様々な気象データ(10日分のデータ)を入力として受け取り、24時間後の気温を予測するモデルを構築する。


In [0]:
!cd ~/Downloads
!mkdir jena_climate
!cd jena_climate
!wget https://s3.amazonaws.com/keras-datasets/jena_climate_2009_2016.csv.zip
!unzip jena_climate_2009_2016.csv.zip

In [0]:
# 今日データセットの中身をみる
import os

#データセットが置かれているディレクトリ
data_dir = ''
fname = os.path.join(data_dir, 'jena_climate_2009_2016.csv')

#データの読み込み
f = open(fname)
data = f.read() 
f.close()


lines = data.split('\n')       #行ごとに改行で区切って
header = lines[0].split(',')   #0行目はheaderとして取り出す
lines = lines[1:]              #1行目以降をlinesに格納する

print(header) #CSVファイルの0行目、Timestamp + 14種類の気象情報を見る。
print(len(lines)) #各時系列データの長さ


In [0]:
#01.01.2009 00:10:00から00:50:00までのデータ(0行目から4行目)を見る
lines[0:5] 

In [0]:
#420551行の時系列データ(ベクトル)をNumPy配列(2次元テンソル)に変換

import numpy as np

float_data = np.zeros((len(lines), len(header) - 1)) #Timestampを消すので-1する
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(',')[1:]] #Timestampを消すので[1:]で0列目を排除
    float_data[i, :] = values

In [0]:
# float_dataが420551行x14列の2次元行列データになっていることを確認
print(float_data.shape)
print(float_data[0:5])

In [0]:
# 気温の時系列データのプロット (2009年から2016年のデータ、8年分)
from matplotlib import pyplot as plt
%matplotlib inline

temp = float_data[:, 1] #気温(T (degC))の列のみ取り出し
plt.plot(range(len(temp)), temp)
plt.show

"""
8年分のデータなので、8個のピーク(夏)が見れる。
"""

In [0]:
# 最初の10日間の気温をデータをプロット (10分おきのデータなので10日分は、(60/10)*24*10 = 1440)
plt.plot(range(1440), temp[:1440])
plt.show()

"""
かなり寒い時期の10日間のデータのよう。最後の4日間は24時間の周期が見れます。
"""

In [0]:
# データの正規化

"""
気温は一般的に-20から+30度程度の値だが、気圧はミリバール単位であり、1000前後の値になります。
そこで、各時系列を個別に正規化し、全ての時系列が同じような尺度の小さな値になるようにする。
訓練データとして最初の20万個のデータを使うので、この部分を正規化する。
"""

#平均値を引く
mean = float_data[:200000].mean(axis=0)
float_data -= mean

#分散で割り算
std = float_data[:200000].std(axis=0)
float_data /= std

In [0]:
# データの準備。(時系列サンプルとそれらの目的値を生成するジェネレータ)
# 入力データのバッチ(samples)とその入力データに対応する目的値(気温)(targets)のタプル(smples, targets)を生成

"""
このデータセットのサンプルはかなり冗長であるため(サンプルNとサンプルN+1のデータがほとんど変わらない)
すべてのサンプルを明示的に割り当てることは非常に無駄であるため、代わりに元のデータを使用してその場でサンプルを生成する。
"""
"""
data: 一つ前のcellで正規化したdata
lookback: 入力データの時刻刻みをいくつ遡るか。10日分のデータを入力して予測に使うので、後で1440 (=6*24*10)に設定する。
delay: 目的値の時刻刻みをいくつ進めるか。今、24時間後のデータ予測を考えているので後で144 (=6*24)に設定する。
min_index, max_index: 抽出する時間刻みの上限と下限を表すdata配列のインデックス。データの一部を検証とテストのために取っておくのに役立つ
shuffle: サンプルをシャッフルするのか、それとも時間の順序で抽出するのかを決める
step: データをサンプリングするときの期間。データ点を1時間ごとに1つ抽出するため6に設定する。
"""

def generator(data, lookback, delay, min_index, max_index, shuffle=False, batch_size=128, step=6):

    if max_index is None:
        max_index = len(data) - delay -1     #最大より1日分手前までのデータしか使わない
    i = min_index + lookback                #最低でも10日分を予測用に確保

    while 1:
        if shuffle:
            rows = np.random.randint(min_index + lookback, max_index, size=batch_size)
            #np.random.randint(a, b, size = c)はaからb-1までの整数がランダムに入ったc次元のベクトルが返ってくる

        else:
            if i + batch_size >= max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index)) #rowsにはiからi+batch_sizeもしくは、iからmax_index - 1までの整数が入る。だいたい前者
            i += len(rows) # len(rows)はだいたいbatch_size

        samples = np.zeros((len(rows),
                            lookback // step,  # lookback // stepは240 (10日分)
                            data.shape[-1]))   # data.shape[-1]は14
        targets = np.zeros((len(rows),))
        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step) # j = 0の時、rows[0] - lookback = min_indexから, rows[0] = i = min_index + lookbackまでstepで刻む
            samples[j] = data[indices]                         # step刻みで10日分のデータを抽出
            targets[j] = data[rows[j] + delay][1]              # targetとして気温のみを抽出
        yield samples, targets


In [0]:
#訓練、検証、テストに使用するデータジェネレータの準備

lookback = 1440   #10日
delay = 144       #1日
step = 6          # 1時間
batch_size = 128

# 訓練データジェネレータ
train_gen = generator(float_data,
                      lookback = lookback,
                      delay = delay,
                      min_index = 0,
                      max_index = 200000,
                      shuffle = True,
                      step = step,
                      batch_size = batch_size)

# 検証データジェネレータ
val_gen = generator(float_data,
                    lookback = lookback,
                    delay = delay,
                    min_index = 200001,
                    max_index = 300000,
                    step = step,
                    batch_size = batch_size)

# テストデータジェネレータ
test_gen = generator(float_data,
                      lookback = lookback,
                      delay = delay,
                      min_index = 300001,
                      max_index = None,
                      step = step,
                      batch_size = batch_size)

# 検証データセット全体を調べるためにval_genから抽出する時間刻みの数 (= 769)
val_steps = (300000 - 200001 - lookback) // batch_size

# テストデータセット全体を調べるためにtest_genから抽出する時間刻みの数 (= 930)
test_steps = (len(float_data) - 300001 - lookback) // batch_size

### 機械学習を行う前に、ベースラインの設定
ここまででデータセットの準備ができた。ここから24時間後の気温の予測を行なっていく。機械学習による解析を行う前に、ベースラインの設定を行う。気温には24時間単位の周期性があると考えることができる。この常識的な感覚を用いて24時間後の気温が現在と同じ気温になると予測してみる。少なくともこの値を超えなければ機械学習を用いる意味がない。平均絶対誤差(MAE(Mean Absolute Error))を指標としてこのアプローチを評価してみる。

In [0]:
def evaluate_naive_method():
    batch_maes = []
    for step in range(val_steps):
        samples, targets = next(val_gen)        # val_genのyieldで返されるデータを順番に抽出
        preds = samples[:, -1, 1]               # 10日分のデータの最後の気温をpredsとして抽出
        mae = np.mean(np.abs(preds - targets))  # 平均絶対誤差を計算
        batch_maes.append(mae)
    print(np.mean(batch_maes))

evaluate_naive_method()

In [0]:
# 上記の値は平均0分散1に正規化されているため、元に戻します。
celsius_mae = 0.29*std[1]
celsius_mae

# 平均絶対誤差はおよそ2.57度になった。機械学習を使ってこれを超える必要がある。

#### 機械学習の基本的アプローチ
RNNなどの複雑なモデルを試す前にまずは基本的なモデルで試してみる。複雑なモデルを導入する根拠にもなる。
データを平坦化した上で2つのDense層で処理します。最後のDense層では活性化関数が設定されていないが回帰ではこれが一般的。損失関数として平均絶対誤差(MAE)を設定しているため、評価に使用するデータと指標は、上で行った常識的なアプローチと全く同じ。そのため、結果を直接比較することができる。

In [0]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

model = Sequential()
model.add(layers.Flatten(input_shape = (lookback //step, float_data.shape[-1]))) #10日分のデータを入力

model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(1))

model.compile(optimizer = RMSprop(), loss ='mae')
hist = model.fit_generator(train_gen,
                           steps_per_epoch=500, #1epochで用いるminibatchの数
                           epochs=20,
                           validation_data=val_gen,
                           validation_steps=val_steps)

"""
これまで使用してきたmodel.fit()は、データがメモリにのるくらいの規模のデータに向いており、model.fit_generator()は、
generatorを使ってバッチ毎にデータを読み込む。メモリ以上のデータを扱うときやバッチ毎に処理をさせるときに使う。
画像を扱う際には、ImageDataGeneratorという便利なgeneratorがKerasでは用意されてる。
画像以外や少し複雑なことをやりたいときは、自分で準備したgeneratorを使うこともできる。
"""

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

loss = hist.history['loss']
val_loss = hist.history['val_loss']

# lossのグラフ
plt.figure(figsize=(8, 5))
plt.plot(range(20), loss, marker='.', label='training_loss')
plt.plot(range(20), val_loss, marker='.', label='validation_loss')
plt.legend(loc='best', fontsize=10)
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')

"""
訓練誤差は下がるが、validation_lossは0.3より悪い値。
ベースラインを超えることができていません。『常識』には機械学習モデルが学習することができない貴重な情報が山ほど含まれている。
"""

#### GRUを使った学習
先ほどのアプローチではデータを平坦化することでデータから時間の概念を取り除いた。
ここでは、データを時系列データとしてそのままの状態で調べてみることにする。
また、LSTMの代わりにGRU(Gated Recurrent Unit)層を使用する。
GRU層はLSTMと同様の原理に基づいているが、少し効率化されているため、LSTMほどコストがかからない。(ただし表現力は劣ります。)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, GRU
from keras.optimizers import RMSprop

model = Sequential()
model.add(GRU(32, input_shape=(None, float_data.shape[-1]))) # このNoneは任意の整数を期待する。ジェネレータにより10日分のデータが生成される為Noneを240にしてもOK
model.add(Dense(1))
model.summary()

model.compile(optimizer=RMSprop(), loss='mae')
hist = model.fit_generator(train_gen,
                           steps_per_epoch=500, #1epochで用いるminibatchの数
                           epochs=20,
                           validation_data=val_gen,
                           validation_steps=val_steps)

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

loss = hist.history['loss']
val_loss = hist.history['val_loss']

# lossのグラフ
plt.figure(figsize=(8, 5))
plt.plot(range(20), loss, marker='.', label='training_loss')
plt.plot(range(20), val_loss, marker='.', label='validation_loss')
plt.legend(loc='best', fontsize=10)
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')

"""
ベースラインを少しだけ上回る結果が出てる。
やや過学習気味なのでドロップアウトなど入れて過学習を抑えると良くなると考えることができる。
"""

#### GRU + dropoutによる学習

過学習を抑えるためにdropoutを導入する。
Kerasには2つのdropout関連パラメータが設定されている。
1つはその層の入力ユニットのdropout率を指定する。
もう1つはrecurrent_dropoutであり、リカレントユニットのdropout率を指定する。全ての時間刻みで同じdropoutマスクがかかる。
dropoutを使ってネットワークを正則化する際は、常に、完全に収束するのにより時間がかかるようになる為、ここではepoch数を2倍にする。

In [0]:
from keras.models import Sequential
from keras import layers
from keras.optimizers import RMSprop

model = Sequential()
model.add(layers.GRU(32,
                     dropout=0.2,
                     recurrent_dropout=0.2,
                     input_shape=(None, float_data.shape[-1])))
model.add(layers.Dense(1))

model.compile(optimizer=RMSprop(), loss='mae')
history = model.fit_generator(train_gen,
                              steps_per_epoch=500,
                              epochs=40,
                              validation_data=val_gen,
                              validation_steps=val_steps)

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

loss = hist.history['loss']
val_loss = hist.history['val_loss']

# lossのグラフ
plt.figure(figsize=(8, 5))
plt.plot(range(40), loss, marker='.', label='training_loss')
plt.plot(range(40), val_loss, marker='.', label='validation_loss')
plt.legend(loc='best', fontsize=10)
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')

#### 1次元の畳み込みを使った学習

ここまでが、RNNを使った時系列データ解析。
教科書(PythonとKerasによるディープラーニング、Francois Chollet著)には他にも、GRUをスタックする方法、
双方向RNNを用いる方法が載っているが、ここではもう一つの時系列データ解析手法である1D convolution層(1次元畳み込み層)を用いた手法を紹介する。
データについて、これまで用いて来た気象データを用い、float_data、train_gen、val_gen、val_stepsを再利用する。

In [0]:
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D, Dense
from keras.optimizers import RMSprop

model = Sequential()
model.add(Conv1D(32, 5, activation='relu', input_shape=(None, float_data.shape[-1]))) 
model.add(MaxPooling1D(3))
model.add(Conv1D(32, 5, activation='relu'))
model.add(MaxPooling1D(3))
model.add(Conv1D(32, 5, activation='relu'))
model.add(GlobalMaxPooling1D()) #2Dconvolutionの時に用いていたFlattenではなくGlobalMaxPooling1Dを用いる。これで3次元出力が2次元になる。
model.add(Dense(1))

model.summary()

In [0]:
model.compile(optimizer=RMSprop(), loss='mae')
hist = model.fit_generator(train_gen,
                           steps_per_epoch=500,
                           epochs=20,
                           validation_data=val_gen,
                           validation_steps=val_steps)

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

loss = hist.history['loss']
val_loss = hist.history['val_loss']

# lossのグラフ
plt.figure(figsize=(8, 5))
plt.plot(range(20), loss, marker='.', label='training_loss')
plt.plot(range(20), val_loss, marker='.', label='validation_loss')
plt.legend(loc='best', fontsize=10)
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')


あまり性能が良くない。これはRNNが時系列データ(データの時間的変化)を捉えるのに対して、
CNNでは時間的な順序に鈍感であり局所的なパターンを認識するため(特に気象データでは最近のデータが意味を持つ)。
CNNでも層を深くすることでより長期間のパターンを認識できるようになるが、気象データのような時間的順序が意味を持つデータには向いていない。
短い繰り返しパターンが続くデータなどの解析では1DCNNが威力を発揮すると考えられる。